## 测试AutoAugmentation的作用

In [1]:
# 自动重新加载外部module，使得修改代码之后无需重新import
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from hdd.device.utils import get_device

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# 设置训练数据的路径
DATA_ROOT = "~/workspace/hands-dirty-on-dl/dataset"
# 设置TensorBoard的路径
TENSORBOARD_ROOT = "~/workspace/hands-dirty-on-dl/dataset"
# 设置预训练模型参数路径
TORCH_HUB_PATH = "~/workspace/hands-dirty-on-dl/pretrained_models"
torch.hub.set_dir(TORCH_HUB_PATH)
# 挑选最合适的训练设备
DEVICE = get_device(["cuda", "cpu"])
print("Use device: ", DEVICE)

Use device:  cuda


### Baseline Data Augmentation

```python
transforms.RandomCrop(size=32, padding=4),
transforms.RandomHorizontalFlip(),
transforms.ToTensor(),
transforms.Normalize(mean=TRAIN_MEAN, std=TRAIN_STD),
```

In [2]:
# 我们提前计算好了训练数据集上的均值和方差
TRAIN_MEAN = [0.49139968, 0.48215827, 0.44653124]
TRAIN_STD = [0.24703233, 0.24348505, 0.26158768]

train_dataset_transforms = transforms.Compose(
    [
        transforms.RandomCrop(size=32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=TRAIN_MEAN, std=TRAIN_STD),
    ]
)
# 加载数据集
train_dataset = datasets.CIFAR10(
    root=DATA_ROOT,
    train=True,
    transform=train_dataset_transforms,
    download=True,
)
val_dataset = datasets.CIFAR10(
    root=DATA_ROOT,
    train=False,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize(TRAIN_MEAN, TRAIN_STD)]
    ),
    download=True,
)
BATCH_SIZE = 128
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
from hdd.models.cnn.resnet import ResnetSmall, resnet34_config
from hdd.train.classification_utils import (
    naive_train_classification_model,
    eval_image_classifier,
)
from hdd.models.nn_utils import count_trainable_parameter


def train_net(
    resnet_config,
    train_dataloader,
    val_dataloader,
    dropout,
    lr,
    weight_decay,
    step_size=40,
    gamma=0.1,
    max_epochs=160,
) -> tuple[ResnetSmall, dict[str, list[float]]]:
    net = ResnetSmall(resnet_config, num_classes=10, dropout=dropout).to(DEVICE)
    criteria = nn.CrossEntropyLoss()
    # SGD的收敛速度远不如Adam好
    # optimizer = torch.optim.SGD(
    #     net.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay
    # )
    optimizer = optim.AdamW(
        net.parameters(), lr=lr, eps=1e-6, weight_decay=weight_decay
    )
    scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer, step_size=step_size, gamma=gamma, last_epoch=-1
    )
    training_stats = naive_train_classification_model(
        net,
        criteria,
        max_epochs,
        train_dataloader,
        val_dataloader,
        DEVICE,
        optimizer,
        scheduler,
        verbose=True,
    )
    return net, training_stats


net, _ = train_net(
    resnet34_config,
    train_dataloader,
    val_dataloader,
    dropout=0.0,
    lr=0.01,
    weight_decay=0,
)

eval_result = eval_image_classifier(net, val_dataloader.dataset, DEVICE)
ss = [result.gt_label == result.predicted_label for result in eval_result]
print(f"#Parameter: {count_trainable_parameter(net)} Accuracy: {sum(ss) / len(ss)}")

Epoch: 1/160 Train Loss: 1.5633 Accuracy: 0.4172 Time: 10.65951  | Val Loss: 1.3756 Accuracy: 0.5193
Epoch: 2/160 Train Loss: 1.0187 Accuracy: 0.6349 Time: 10.68677  | Val Loss: 0.9361 Accuracy: 0.6767
Epoch: 3/160 Train Loss: 0.7805 Accuracy: 0.7261 Time: 10.45274  | Val Loss: 0.7739 Accuracy: 0.7284
Epoch: 4/160 Train Loss: 0.6293 Accuracy: 0.7789 Time: 10.32677  | Val Loss: 0.6849 Accuracy: 0.7699
Epoch: 5/160 Train Loss: 0.5348 Accuracy: 0.8125 Time: 10.46105  | Val Loss: 0.6484 Accuracy: 0.7899
Epoch: 6/160 Train Loss: 0.4730 Accuracy: 0.8359 Time: 10.63133  | Val Loss: 0.4966 Accuracy: 0.8314
Epoch: 7/160 Train Loss: 0.4143 Accuracy: 0.8584 Time: 10.47751  | Val Loss: 0.4930 Accuracy: 0.8345
Epoch: 8/160 Train Loss: 0.3785 Accuracy: 0.8675 Time: 10.39104  | Val Loss: 0.4361 Accuracy: 0.8524
Epoch: 9/160 Train Loss: 0.3345 Accuracy: 0.8840 Time: 10.60739  | Val Loss: 0.4769 Accuracy: 0.8496
Epoch: 10/160 Train Loss: 0.3101 Accuracy: 0.8922 Time: 10.60040  | Val Loss: 0.3983 Accura

### [AutoAugmentation](https://arxiv.org/abs/1805.09501v1)

```python
transforms.RandomCrop(size=32, padding=4),
transforms.RandomHorizontalFlip(),
CIFAR10Policy(),
transforms.ToTensor(),
transforms.Normalize(mean=TRAIN_MEAN, std=TRAIN_STD),
```

In [6]:
from hdd.data_util.auto_augmentation import CIFAR10Policy

train_dataset_transforms = transforms.Compose(
    [
        transforms.RandomCrop(size=32, padding=4),
        transforms.RandomHorizontalFlip(),
        CIFAR10Policy(),
        transforms.ToTensor(),
        transforms.Normalize(mean=TRAIN_MEAN, std=TRAIN_STD),
    ]
)
# 加载数据集
train_dataset = datasets.CIFAR10(
    root=DATA_ROOT,
    train=True,
    transform=train_dataset_transforms,
    download=True,
)
val_dataset = datasets.CIFAR10(
    root=DATA_ROOT,
    train=False,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize(TRAIN_MEAN, TRAIN_STD)]
    ),
    download=True,
)
BATCH_SIZE = 128
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
)

net, _ = train_net(
    resnet34_config,
    train_dataloader,
    val_dataloader,
    dropout=0.0,
    lr=0.01,
    weight_decay=0,
)

eval_result = eval_image_classifier(net, val_dataloader.dataset, DEVICE)
ss = [result.gt_label == result.predicted_label for result in eval_result]
print(f"#Parameter: {count_trainable_parameter(net)} Accuracy: {sum(ss) / len(ss)}")

Files already downloaded and verified
Files already downloaded and verified
Epoch: 1/160 Train Loss: 1.8869 Accuracy: 0.2928 Time: 10.49294  | Val Loss: 1.4959 Accuracy: 0.4509
Epoch: 2/160 Train Loss: 1.3630 Accuracy: 0.5111 Time: 10.45894  | Val Loss: 1.0653 Accuracy: 0.6243
Epoch: 3/160 Train Loss: 1.1004 Accuracy: 0.6115 Time: 10.51756  | Val Loss: 0.8691 Accuracy: 0.7004
Epoch: 4/160 Train Loss: 0.9250 Accuracy: 0.6752 Time: 10.50109  | Val Loss: 0.7017 Accuracy: 0.7632
Epoch: 5/160 Train Loss: 0.8099 Accuracy: 0.7151 Time: 10.45488  | Val Loss: 0.6079 Accuracy: 0.7844
Epoch: 6/160 Train Loss: 0.7292 Accuracy: 0.7440 Time: 10.71078  | Val Loss: 0.5311 Accuracy: 0.8152
Epoch: 7/160 Train Loss: 0.6726 Accuracy: 0.7669 Time: 10.55842  | Val Loss: 0.5624 Accuracy: 0.8097
Epoch: 8/160 Train Loss: 0.6267 Accuracy: 0.7826 Time: 10.49996  | Val Loss: 0.5189 Accuracy: 0.8199
Epoch: 9/160 Train Loss: 0.5896 Accuracy: 0.7939 Time: 10.54540  | Val Loss: 0.4596 Accuracy: 0.8390
Epoch: 10/160 T